In [12]:
from elasticsearch import Elasticsearch

In [29]:
es = Elasticsearch("https://192.168.1.130:9200", 
                   basic_auth=("elastic", "CIyX=*SG=U79tDbkd4Fw"),
                   verify_certs=False
                   )

es.ping()

/Users/remygodin/Desktop/Semester2/DSE/project/elastic_test/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True

## Prepare the data

## convert relevant field to vector using BERT model

In [30]:
import pandas as pd
df = pd.read_csv("../Cup.Russia.Matches.csv", sep=";")
df.head()

,MatchID,Time (Brazil),Date,Stage,Stadium,City,Home Team,Home Team Goals,Away Team Goals,Away Team,Win Conditions,Penalty,Win,Total Goals,Attendance
0,0,12:00,2018-06-14,Group A,Luzhniki Stadium,Moscow,Russia,5,0,Saudi Arabia,Normal Time,NaN,Russia,5,78011
1,1,09:00,2018-06-15,Group A,Ekaterinburg Arena,Ekaterinburg,Egypt,0,1,Uruguay,Normal Time,NaN,Uruguay,1,27015
2,2,12:00,2018-06-15,Group B,Saint Petersburg Stadium,St. Petersburg,Morocco,0,1,IR Iran,Normal Time,NaN,IR Iran,1,62548
3,3,15:00,2018-06-15,Group B,Fisht Stadium,Sochi,Portugal,3,3,Spain,Normal Time,NaN,Empate,6,43866
4,4,07:00,2018-06-16,Group C,Kazan Arena,Kazan,France,2,1,Australia,Normal Time,NaN,France,3,41279


In [31]:
df.isna().value_counts()

MatchID  Time (Brazil)  Date   Stage  Stadium  City   Home Team  Home Team Goals  Away Team Goals  Away Team  Win Conditions  Penalty  Win    Total Goals  Attendance
False    False          False  False  False    False  False      False            False            False      False           True     False  False        False         60
                                                                                                                              False    False  False        False          4
Name: count, dtype: int64

In [32]:
df.fillna("None", inplace=True)

## Convert relevant field to Vector using BERT model

## 

In [33]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [34]:
df["DescriptionVector"] = df["Home Team"].apply(lambda x: model.encode(x))

In [35]:
df.head()


,MatchID,Time (Brazil),Date,Stage,Stadium,City,Home Team,Home Team Goals,Away Team Goals,Away Team,Win Conditions,Penalty,Win,Total Goals,Attendance,DescriptionVector
0,0,12:00,2018-06-14,Group A,Luzhniki Stadium,Moscow,Russia,5,0,Saudi Arabia,Normal Time,None,Russia,5,78011,"[0.046087243, -0.0037010114, -0.01822217, -0.0..."
1,1,09:00,2018-06-15,Group A,Ekaterinburg Arena,Ekaterinburg,Egypt,0,1,Uruguay,Normal Time,None,Uruguay,1,27015,"[0.047417443, 0.06572734, -0.030754808, -0.056..."
2,2,12:00,2018-06-15,Group B,Saint Petersburg Stadium,St. Petersburg,Morocco,0,1,IR Iran,Normal Time,None,IR Iran,1,62548,"[0.02808081, -0.025342938, -0.04487486, -0.068..."
3,3,15:00,2018-06-15,Group B,Fisht Stadium,Sochi,Portugal,3,3,Spain,Normal Time,None,Empate,6,43866,"[0.02427361, 0.037476208, -0.023086928, -0.048..."
4,4,07:00,2018-06-16,Group C,Kazan Arena,Kazan,France,2,1,Australia,Normal Time,None,France,3,41279,"[0.03814006, 0.086826846, -0.0024928255, -0.03..."


In [36]:
from indexMapping import indexMapping

es.indices.create(index="dse_project_v2", mappings=indexMapping)

/Users/remygodin/Desktop/Semester2/DSE/project/elastic_test/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'dse_project_v2'})

In [37]:
record_list = df.to_dict("records")


In [40]:
for record in record_list:
    try:
        es.index(index="dse_project_v2", document=record, id=record["MatchID"])
    except Exception as e:
        print(e)

/Users/remygodin/Desktop/Semester2/DSE/project/elastic_test/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/remygodin/Desktop/Semester2/DSE/project/elastic_test/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/remygodin/Desktop/Semester2/DSE/project/elastic_test/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.130'. Adding certificate verification is strongly advised. See: ht

In [41]:
es.count(index="dse_project_v2")


/Users/remygodin/Desktop/Semester2/DSE/project/elastic_test/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'count': 64, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [42]:
input_keyword = "France game at home"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 500, 
}

res = es.knn_search(index="dse_project_v2", knn=query , source=["Win","Home Team"])
res["hits"]["hits"]

/Users/remygodin/Desktop/Semester2/DSE/project/elastic_test/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.130'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/var/folders/rc/_2402hms0832c05bh3s9z6sh0000gn/T/ipykernel_63955/456895625.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="dse_project_v2", knn=query , source=["Win","Home Team"])


[{'_index': 'dse_project_v2',
  '_id': '4',
  '_score': 0.7401401,
  '_source': {'Home Team': 'France', 'Win': 'France'}},
 {'_index': 'dse_project_v2',
  '_id': '21',
  '_score': 0.7401401,
  '_source': {'Home Team': 'France', 'Win': 'France'}}]